In [1]:
# Standard
# None

# Pip
# None

# Custom
from api_nlp.sentence_generator.bundled_gap_generator import *


def main(corpus, target):
    # Korpus, aus dem Sätze gezogen werden sollen, und Target festlegen sowie leeres Bundle initiieren
    print("Initialisierung...")
    bundle = []
    # Sätze, die das Target enthalten, als Liste in Liste schreiben
    corpus_list = []
    for line in corpus:
        if target in line:
            # Target markieren
            line = line.replace(target, "<" + target + ">")
            list = line.split()
            # ... und als Liste an unsere Liste von in Frage kommenden Sätzen anhängen
            corpus_list.append(list)

    # Seed sentence auswählen
    seed_id, seed_sentence = choose_seed(target, corpus_list)

    # Satz zum Bundle hinzufügen
    bundle.append(seed_sentence)
    # Diesen Satz aus corpus_list entfernen (denn wir wollen ja keinen Satz doppelt im Bundle haben)
    corpus_list.remove(corpus_list[seed_id])

    print("Generiere Distraktoren mit Fastsubs...")
    # Von Fastsubs distractors generieren lassen
    distractor_list = fastsubs_distractor_generator(
        fastsubs_instance, seed_sentence, target
    )

    print("...")
    print("Unser erster Satz:")
    print("'" + " ".join(seed_sentence) + "'")
    print("Disambiguation level: " + str(disamb(bundle, target, distractor_list)))

    print("...")
    print("Jetzt fangen wir an, weitere Sätze zu suchen.")
    print("...")

    # Hier suchen wir mithilfe einer while-Schleife immer den nächsten Satz aus, der unser disambiguation level optimiert.
    count = 1
    while count < 4:
        best_sentence_id = best_next_sentence(
            corpus_list, bundle, target, distractor_list
        )
        if best_sentence_id != -1:
            bundle.append(corpus_list[best_sentence_id])
            corpus_list.remove(corpus_list[best_sentence_id])
            print("Satz Nr. " + str(count + 1) + " gefunden!")
            print(
                "Disambiguation level: " + str(disamb(bundle, target, distractor_list))
            )
        else:
            bundle.append(
                ["ERROR: target_prob/max_other_prob = 0. Noch einmal versuchen?"]
            )
            break
        count = count + 1

    # output
    output(bundle, target)


if __name__ == "__main__":
    corpus = open(
        "data/incoming_text_data/sentences.txt",
        "r",
    )
    target = "pears"
    main(corpus, target)


Loading the LM will be faster if you build a binary file.
Reading /Users/christopherchandler/code_repos/christopher-chandler/Python/nlp/rub/bundled_gap_filling/data/language_model/en-70k-0.2-pruned.lm
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
*The ARPA file is missing <unk>.  Substituting log10 probability -100.
***************************************************************************************************


Initialisierung...
Generiere Distraktoren mit Fastsubs...
...
Unser erster Satz:
'During the autumn harvest, families enjoyed picking <pears> from the trees.'
Disambiguation level: -7.62167832897076
...
Jetzt fangen wir an, weitere Sätze zu suchen.
...
Satz Nr. 2 gefunden!
Disambiguation level: -10.466113693152575
Satz Nr. 3 gefunden!
Disambiguation level: 1000
Satz Nr. 4 gefunden!
Disambiguation level: 1000
...
Hier ist die fertige Bundled Gap Filling Aufgabe:

'During the autumn harvest, families enjoyed picking <pears> from the trees.'
'The fruit bowl on the kitchen table displayed a mix of <pears> in different shapes and colors.'
'Sarah baked a delicious pear and almond tart with the ripe <pears> from her garden.'
'As they strolled through the farmers' market, Emily and David picked up apples, <pears>, and oranges for their upcoming picnic.'
